In [2]:
%load_ext autoreload
%autoreload 2
import numpy as np
import casadi as ca
from decision_vars import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
# Test named dict
d = NamedDict('test/', {'a':3})
print(d['a'])
print(d.get('test/a'))
print(d.keys())
for k in d.keys():
    print(d.get(k))


3
3
dict_keys(['test/a'])
3


In [3]:
# Test decision vars
dvs = DecisionVarDict(attr_names = ['lb'], name = 'funtimes/')
dvs.add_vars({'x':[1.0, 2.0]}, lb={'x':[0, 0]})
print(dvs)
print(dvs['funtimes/x'])
print(dvs.get('funtimes/x'))
print(dvs.get_from_shortname('x'))
dvs2 = DecisionVarDict(attr_names = ['lb'], name = 'ouble/')
dvs2.add_vars(init={'x':2.0}, lb={'y':3.0})
dvs += dvs2
print(dvs.vectorize('lb'))
d2 = dvs.dictize(np.array([0.1, 0.3, 0.5]))
print(d2)
print(d2.keys())


***** Decision Vars, name: funtimes/ *****
Attributes: ['lb']
Vars: 
  funtimes/x: [x_0, x_1], shape: (2, 1)

[x_0, x_1]
[x_0, x_1]
[x_0, x_1]


AttributeError: 'DecisionVarDict' object has no attribute 'vectorize'

In [4]:
a = DecisionVarDict(attr_names = ['lb'], name = '')
a.add_vars({'q':[1.0, 2.0]}, lb={'q':[0, 0]})

b = DecisionVarDict(attr_names = ['lb'], name = 'free/')
b.add_vars({'q':[1.0, 2.0]}, lb={'q':[0, 0]})

a += b
print(a)
print(a['q'][0] is b['free/q'][0])

print(b.get_vars('init'))


print(a.get_vectors())
print(a.get_vars())
print(a.get_vars('lb'))
print(a.vectorize())
print(a.dictize(a.vectorize()))
print(a.clone_and_extend(2).keys())


***** Decision Vars, name:  *****
Attributes: ['lb']
Vars: 
  q: [q_0, q_1], shape: (2, 1)
  free/q: [q_0, q_1], shape: (2, 1)

False
{'free/q': array([1., 2.])}
()
{'q': SX([q_0, q_1]), 'free/q': SX([q_0, q_1])}
{'q': array([0, 0]), 'free/q': array([0, 0])}


AttributeError: 'DecisionVarDict' object has no attribute 'vectorize'

In [5]:
a = DecisionVarDict(attr_names = ['lb'], name = '')
a.add_vars({'q':[1.0, 2.0]}, lb={'q':[0, 0]})

b = DecisionVarDict(attr_names = ['lb'], name = 'free/')
b.add_vars({'q':[1.0, 2.0]}, lb={'q':[0, 0]})

a += b

print(a.clone_and_extend(3))
print(a.clone_and_vectorize('xi'))
print(a.clone_and_vectorize('xi').clone_and_extend(3).get_vars('lb'))

***** Decision Vars, name:  *****
Attributes: ['lb']
Vars: 
  q: 
[[q_0, q_2, q_4], 
 [q_1, q_3, q_5]], shape: (2, 3)
  free/q: 
[[free/q_0, free/q_2, free/q_4], 
 [free/q_1, free/q_3, free/q_5]], shape: (2, 3)

***** Decision Vars, name:  *****
Attributes: ['lb']
Vars: 
  xi: [q_0, q_1, q_0, q_1], shape: (4, 1)

{'xi': array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])}


In [6]:
from robot import *
rob = Robot('config/franka.urdf', name = 'test/')
print(rob._state)
rob.build_step(0.1)
print(rob.step)
print(rob._state.clone_and_extend(2))

***** Decision Vars, name: test/ *****
Attributes: []
Vars: 
  test/q: [q_0, q_1, q_2, q_3, q_4, q_5, q_6], shape: (7, 1)
  test/dq: [dq_0, dq_1, dq_2, dq_3, dq_4, dq_5, dq_6], shape: (7, 1)

step:(test/q[7],test/dq[7],M_inv[7x7],tau_input[7])->(q[7],dq[7],cost) SXFunction
***** Decision Vars, name: test/ *****
Attributes: []
Vars: 
  test/q: 
[[q_0, q_7], 
 [q_1, q_8], 
 [q_2, q_9], 
 [q_3, q_10], 
 [q_4, q_11], 
 [q_5, q_12], 
 [q_6, q_13]], shape: (7, 2)
  test/dq: 
[[dq_0, dq_7], 
 [dq_1, dq_8], 
 [dq_2, dq_9], 
 [dq_3, dq_10], 
 [dq_4, dq_11], 
 [dq_5, dq_12], 
 [dq_6, dq_13]], shape: (7, 2)



In [7]:
from contact import Contact
pars = {'pos': [0, 0, 0],
        'stiff': [0, 10, 10],
        'rest':[0, 0, 0.0]}
c = Contact('friendo/', pars)
q = ca.SX.sym('q',7)
p, R = rob.fwd_kin(q)
print(c.get_force({'p':ca.DM([0.5, 0.3, -0.5]), 'R':ca.DM.ones(3)}))

c2 = Contact('c2/',pars, ['stiff'], {'cov_init':{}, 'proc_noise':{}})
print(c2.get_force({'p':ca.DM([0.5, 0.3, -0.5]), 
                    'R':ca.DM.ones(3), 
                    'c2/stiff':ca.DM([10, 0, 0])}))

AttributeError: 'Contact' object has no attribute '_Contact__F_fn'

In [24]:
# Test convergence
from robot import Robot
from contact import Contact

par1 = {'pos': [0, 0, 0.0],
        'stiff': [0, 0, 500],
        'rest':[0, 0, 0.7]}
par2 = {'pos': [0, 0, 0],
        'stiff': [0, 200, 0],
        'rest': [0, 0.2, 0]}
c1 = Contact('c1/', par1, [], {'cov_init':{}, 'proc_noise':{}})
c2 = Contact('c2/', par2, [], {'cov_init':{}, 'proc_noise':{}})

rob = Robot('config/franka.urdf', subsys = [c1, c2], name = 'test/', visc_fric=60)
#rob = Robot('config/franka.urdf', subsys = [c1], name = '', visc_fric=60)

rob.build_step(0.05)
print(rob.get_tau_ext(0.8*ca.DM.ones(7), ca.DM.zeros(7)))
# test that it converges as expected 
q = np.ones(7)
dq = np.zeros(7)
tau = np.zeros(7)
state = NamedDict(rob.name, {'q':q, 'dq':dq})
inp_args = dict(tau_input=tau, M_inv=rob.inv_mass_fn(state['q']))
for sys in rob._subsys:
    print(sys._tau_fn)
for _ in range(500):
    inp_args.update(state)
    res = rob.step(**inp_args)
    state['q'] = res['q']
    state['dq'] = res['dq']
    res_dict = rob.get_ext_state(state)
    #print(res_dict['c1/tau'].T)
    #print(res_dict['p'].T)
    #print(res_dict['dx'].T)
res_dict = rob.get_ext_state(state)
print(res_dict['p'])

[3.52, 9.944, 3.52, -0.6215, 3.52, -9.944, 0]
tau:(q[7],dq[7])->(tau[7x1,5nz]) SXFunction
tau:(q[7],dq[7])->(tau[7x1,6nz]) SXFunction
dict_keys(['test/q', 'test/dq'])
dict_keys(['test/q', 'test/dq'])
dict_keys(['test/q', 'test/dq'])
dict_keys(['test/q', 'test/dq'])
dict_keys(['test/q', 'test/dq'])
dict_keys(['test/q', 'test/dq'])
dict_keys(['test/q', 'test/dq'])
dict_keys(['test/q', 'test/dq'])
dict_keys(['test/q', 'test/dq'])
dict_keys(['test/q', 'test/dq'])
dict_keys(['test/q', 'test/dq'])
dict_keys(['test/q', 'test/dq'])
dict_keys(['test/q', 'test/dq'])
dict_keys(['test/q', 'test/dq'])
dict_keys(['test/q', 'test/dq'])
dict_keys(['test/q', 'test/dq'])
dict_keys(['test/q', 'test/dq'])
dict_keys(['test/q', 'test/dq'])
dict_keys(['test/q', 'test/dq'])
dict_keys(['test/q', 'test/dq'])
dict_keys(['test/q', 'test/dq'])
dict_keys(['test/q', 'test/dq'])
dict_keys(['test/q', 'test/dq'])
dict_keys(['test/q', 'test/dq'])
dict_keys(['test/q', 'test/dq'])
dict_keys(['test/q', 'test/dq'])
dict_key

In [13]:
from robot import Robot
from contact import Contact
par1 = {'pos': [0, 0, 0.0],
        'stiff': [0, 0, 100],
        'rest':[0, 0, 0.2]}
par2 = {'pos': [0, 0, 0],
        'stiff': [0, 100, 0],
        'rest': [0, 0.5, 0]}
c1 = Contact('c1/', par1, ['stiff'], {'cov_init':{}, 'proc_noise':{}})
c2 = Contact('c2/', par2, [], {'cov_init':{}, 'proc_noise':{}})

rob = Robot('config/franka.urdf', subsys = [c1, c2])


In [14]:
# Check that the spawner works
from helper_fns import spawn_models
models, _, _ = spawn_models('config/franka.yaml', 'config/attrs.yaml', 'config/contact.yaml', sym_vars = [])
for name, model in models.items():
    model.build_step(step_size=0.1)
    n_in = model.step.size_in(0)
    res = model.step(np.zeros(7), np.zeros(7), np.eye(7), np.zeros(7))
    print(f"{name} has step fn of {model.step}")
    print(f"  which evals to {res}")


free has step fn of step:(q[7],dq[7],M_inv[7x7],tau_input[7])->(q[7],dq[7],cost) SXFunction
  which evals to (DM([0, 0, 0, 0, 0, 0, 0]), DM([0, 0, 0, 0, 0, 0, 0]), DM(0))
point has step fn of step:(q[7],dq[7],M_inv[7x7],tau_input[7])->(q[7],dq[7],cost) SXFunction
  which evals to (DM([-4.81091e-16, -8.80012, 5.60174e-16, 4.11068, -7.0516e-16, -1.58788, 0]), DM([-4.81091e-15, -88.0012, 5.60174e-15, 41.1068, -7.0516e-15, -15.8788, 0]), DM(0))


In [3]:
# Check the switched mdel 
from helper_fns import spawn_switched_models, spawn_models
robots_sw, contacts = spawn_switched_models(robot_path = "config/franka.yaml",
                                         contact_path = "config/contact_test.yaml",
                                         attr_path  = "config/attrs.yaml", 
                                         mode = "point")
_, robots, contacts = spawn_models(robot_path = "config/franka.yaml",
                                         contact_path = "config/contact_test.yaml",
                                         attr_path  = "config/attrs.yaml")


rsw = robots_sw['point']
r = robots['point']
rsw.build_step(0.05)
r.build_step(0.05)

q0 = np.ones(7)
dq0 = -0.5*np.ones(7)
state = {'point/q': q0,
         'point/dq': dq0,
         'M_inv': r.inv_mass_fn(q0),
         'imp_stiff':0*ca.DM.ones(3),
         'imp_rest': [0,0,0]}
xsw = rsw.get_ext_state(state)
x = r.get_ext_state(state)
print(x['p'])
print(x['dx'])
print(xsw['F_ext'])
print(x['F_ext'])
rob = r
print(rob.step)
for _ in range(300):
    res = rob.step.call(state)
    print(res['cost'])
    state['point/q'] = res['q']
    state['point/dq'] = res['dq']
    res_dict = rob.get_ext_state(state)    
    #print(res_dict['p'][0])
    #print(res_dict['F_ext'][0])

[[0.30016132]
 [0.18034758]
 [0.8164567 ]]
[[-0.1651301 ]
 [-0.05292904]
 [ 0.21195034]]
[[0.]
 [0.]
 [0.]]
[[-2.064528]
 [ 0.      ]
 [ 0.      ]]
step:(point/q[7],point/dq[7],M_inv[7x7],imp_stiff[3],imp_rest[3])->(q[7],dq[7],cost) SXFunction
[-0.40507161]
[0.39795878]
[0.76013565]
[0.90142259]
[0.93419406]
[0.91497654]
[0.87205093]
[0.81952002]
[0.76436673]
[0.7099876]
[0.6579677]
[0.60897718]
[0.56322667]
[0.52069955]
[0.48127089]
[0.44476814]
[0.41100204]
[0.37978212]
[0.35092413]
[0.32425353]
[0.29960676]
[0.27683147]
[0.25578623]
[0.23634003]
[0.2183716]
[0.20176879]
[0.18642792]
[0.17225314]
[0.15915587]
[0.14705424]
[0.1358726]
[0.12554103]
[0.11599493]
[0.1071746]
[0.09902489]
[0.09149482]
[0.08453728]
[0.07810875]
[0.07216903]
[0.06668094]
[0.06161017]
[0.05692497]
[0.05259603]
[0.04859627]
[0.04490067]
[0.04148608]
[0.03833116]
[0.03541614]
[0.0327228]
[0.03023427]
[0.02793499]
[0.02581056]
[0.02384768]
[0.02203408]
[0.02035839]
[0.01881014]
[0.01737963]
[0.0160579]
[0.01483